# NAVER TRENDS CRAWLING

* 공식 API 제공 (ID/SECRET 발급)
* 각 영화마다 키워드 설정하여 트랜드 조회 가능
* 기간별, 성별, 연령별, 채널별 조회 가능
* 동시 조회 가능 검색어 5개

[Reference](https://brunch.co.kr/@sukhyun9673/13)

### Load Package

In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime
import urllib.request
import json

### Define Class

In [ ]:
class NaverTrend:
    def __init__(self):
        """
        NaverTrend Class
        """
        self.min_movie = []
        self.max_movie = []

    def set_api_info(self, api_info):
        self.client_id = api_info["client_id"]
        self.client_secret = api_info["client_secret"]
        self.url = api_info["url"]
        return

    def load_data(self, movie_data):
        self.movie_data_full = movie_data
        self.movie_list_full = movie_data.iloc[:,0]
        return

    def _define_body(self, movie_list):
        """
        movie_list: list
        TREND 조회 요청에 필요한 Body 파트를 정의한다.
         - 상세 조건 parameter로 받을 수 있도록 수정 필요
         - movie_list, keyword_list 쌍으로 받을 수 있도록 수정 필요 (DataFrame, Nested List, Two Series, ...)
        """
        keywords_list = ["{\"groupName\": \"" + movie + "\", \"keywords\": [\"" + movie + "\"]}" for movie in movie_list]
        body ="{\"startDate\":\"2018-12-01\", \"endDate\":\"2020-02-28\", \"timeUnit\":\"date\", \"keywordGroups\":" + "[" + ", ".join(keywords_list) + "]}"
        return(body)

    def _get_response_body(self, body):
        """
        body: string generated by _define body()
        Naver API로부터 트렌드 조회 결과를 받아온다.
        """
        request = urllib.request.Request(self.url)
        request.add_header("X-Naver-Client-Id", self.client_id)
        request.add_header("X-Naver-Client-Secret", self.client_secret)
        request.add_header("Content-Type", "application/json")
        response = urllib.request.urlopen(request, data = body.encode("utf-8"))
        rescode = response.getcode()
        if(rescode==200):
            response_str = response.read().decode("utf-8") # repsonse body string
            response_dict = json.loads(response_str) # response body dict
            return(response_dict)
        else:
            print("Error Code:" + rescode)
    
    def _response_to_df(self, response_dict):
        """
        response_dict: dictionary generated by _get_response_body()
        response_body_dict를 DataFrame으로 변환한다.
        """
        trend_list = []
        for i in range(len(response_dict["results"])): # Need to find for loop alternatives.
            ttl = response_dict["results"][i]["title"]
            data = response_dict["results"][i]["data"]
            
            time = np.array([datetime.strptime(i["period"], "%Y-%m-%d") for i in data])
            value = np.array([j["ratio"] for j in data])
            title = np.repeat(ttl, len(value), axis=0)
            
            trend = pd.DataFrame({"Keyword": title, "Time": time, "TrendIndex": value})
            trend_list.append(trend)
        response_df = pd.concat(trend_list).reset_index(drop=True)
        return(response_df)

    def _recursive_search(self, init_cand, find_max=True):
        """
        init_cand: initial min/max candidates
        전체 리스트 중 검색 횟수가 최대/최소인 영화를 찾을 때까지 5개씩 TREND를 조회한다.
        """
        def _search(cand):
            n_movie = len(cand)
            rng = range(0, n_movie, 5)

            cand_subset = []
            for i, idx in zip(rng, range(len(rng))):
                movie_subset = [*cand[i:i+5]]
                trend_subset = self.get_trend(movie_subset)
                
                cand_idx = trend_subset['TrendIndex'].idxmax() if find_max == True else trend_subset['TrendIndex'].idxmin()
                cand_in_five = trend_subset.iloc[cand_idx]["Keyword"]  
                cand_subset.append(cand_in_five)
                print(cand_in_five, " in ", movie_subset)
            return(cand_subset)
        
        print("\n============================== CANDIDATES ==============================")
        cand = _search(init_cand)
        if len(cand) != 1:
            return(self._recursive_search(init_cand[init_cand.isin([*cand])]))
        else:
            return(cand)

    def get_trend(self, target, along_with = False):
        """
        특정 영화의 TREND를 조회한다. along_with = True일 경우, min/max moive와 함께 조회하여 스케일을 조정한다.
        """
        target_input = target
        if along_with == True:
            target = set([*target, *self.min_movie, *self.max_movie]) 
            print("Scale Info: max_movie: {0} | min_movie: {1} | target: {2}".format(self.max_movie, self.min_movie, target_input))
        body = self._define_body(set([*target]))
        response_dict_subset = self._get_response_body(body)
        response_subset = self._response_to_df(response_dict_subset)
        target_trend = response_subset[response_subset.Keyword.isin([*target_input])].reset_index(drop=True)
        return(target_trend)

    def set_movie(self, max = [], min = []):
        self.max_movie = self._recursive_search(self.movie_list_full, find_max = True) if max == True else max        
        self.min_movie = self._recursive_search(self.movie_list_full, find_max = False) if min == True else min
        return